In [1]:
import pandas as pd
import numpy as np
# function to check distribution of labels
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore') 

def plot_values(dataset, attribute):
    plt.figure()
    sns.countplot(x=attribute, data=dataset, palette='hls')


---------------------------
# import about data

### The data from Log2 and Log3 are combined in one document HVAC.csv file

### The attributes of the file are renamed to avoid any duplication in term of feature name
----------------------------

In [2]:
#read Data

data = pd.read_csv('HVAC.csv', encoding="utf-8-sig")
data.head()

,Hour of the year,Hour of the day,Tamb,A_Tz1,A_Tz2,A_Tz3,A_Tz4,B_Tz1,B_Tz2,B_Tz3,...,BBB_z1,BBB_z2,BBB_z3,BBB_z4,CCC_z1,CCC_z2,CCC_z3,CCC_z4,Total power usage,Label
0,5094.791,6.791,28.227,20.197,21.169,18.528,20.274,21.746,20.358,20.229,...,0.038,-0.133,-0.174,0.283,-0.174,-0.057,-0.184,0.143,602564.9294,0
1,5094.806,6.806,28.239,20.061,21.085,18.457,20.206,22.614,20.271,19.972,...,0.150,-0.142,-0.206,0.277,-0.193,-0.067,-0.208,0.139,554338.7444,0
2,5094.821,6.821,28.251,20.185,21.182,18.544,20.272,22.274,20.369,20.122,...,0.099,-0.138,-0.194,0.278,-0.157,-0.060,-0.171,0.139,562766.6021,0
3,5094.836,6.836,28.263,20.155,21.120,18.468,20.195,21.220,20.303,20.200,...,-0.025,-0.136,-0.174,0.277,-0.154,-0.057,-0.163,0.137,598766.8626,0
4,5094.851,6.851,28.275,20.087,21.087,18.437,20.176,21.521,20.267,20.080,...,0.015,-0.138,-0.187,0.277,-0.187,-0.061,-0.198,0.137,538247.8916,0


In [3]:
data['Label'].value_counts()

0    37722
1     3279
Name: Label, dtype: int64

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVR
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB


from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, cohen_kappa_score, accuracy_score, f1_score, recall_score, balanced_accuracy_score

import warnings
warnings.filterwarnings('ignore') 

In [5]:
from timer_py import Timer
t = Timer()


def plot_roc_curve(fper, tper):  
    plt.plot(fper, tper, color='orange', label='ROC')
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
   # plt.title('')
    plt.legend()
    plt.show()

def crossCheck(clf, X_train, X_test, y_train, y_test):
    from sklearn import metrics  
    clf.fit(X_train, y_train)
    t.start()
    y_pred = clf.predict(X_test)
    
    elapse = t.elapsed(print=False)
    t.stop(print=False) # A few seconds later
    
    Acc = metrics.accuracy_score(y_test, y_pred)
    Recal = metrics.recall_score(y_test, y_pred)#, average='weighted')
    Prec = metrics.precision_score(y_test, y_pred)#, average='weigh')
    F1 = metrics.f1_score(y_test, y_pred)
    #plot_roc_curve(fpr, tpr)
    #print("\nConfusion Matrix")
    #print(confusion_matrix(y_test, y_pred))
    #fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    #AUC = metrics.auc(fpr, tpr)
    yAUC_pred = clf.predict_proba(X_test)
    AUC = metrics.roc_auc_score(y_test, yAUC_pred[:,1])
    

    return AUC, Acc, Recal, Prec, F1, elapse



def classifiers():
    from xgboost import XGBClassifier
    # scikit-learn k-fold cross-validation
    from numpy import array
    from sklearn.model_selection import KFold
    from sklearn import metrics  
    from sklearn.metrics import confusion_matrix

    from sklearn.ensemble import RandomForestClassifier
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.linear_model import LogisticRegression
    from sklearn.naive_bayes import GaussianNB
    from sklearn.ensemble import AdaBoostClassifier
    
    from sklearn.tree import DecisionTreeClassifier
    
    import lightgbm as lgb
    from catboost import CatBoostClassifier

    DT = DecisionTreeClassifier()
    CB = CatBoostClassifier(
        iterations=5, 
        learning_rate=0.8, 
        #loss_function='CrossEntropy'    
    )

    LG= lgb.LGBMClassifier()

    NB = GaussianNB()
    AB = AdaBoostClassifier()
    LR = LogisticRegression(penalty='l2', random_state=777, max_iter=10000, tol=10)
    RF = RandomForestClassifier(random_state = 1,
                                      n_estimators = 100,
                                      max_depth = 15, 
                                      min_samples_split = 5,  min_samples_leaf = 1)
    GBI = GradientBoostingClassifier(learning_rate=0.05, max_depth=3,
                                     max_features=0.5,
                                     random_state=42)
    XG = XGBClassifier(eval_metric='mlogloss')

    print("Classifier:    AUC        Accuracy     Recall   Prec     F1     time-Elapsed")
    print('-----------------------------------------------------------------------------')

    AUC, Acc, Recal, Prec, F1, E = crossCheck(LR,  X_train, X_test, y_train, y_test)
    print("Logistric Reg:  %0.4f    %0.4f       %0.4f    %0.4f   %0.4f   %0.4f" % (AUC, Acc,Recal,Prec,F1, E))

    AUC, Acc, Recal, Prec, F1, E =  crossCheck(DT,  X_train, X_test, y_train, y_test)
    print("Decision Tree:  %0.4f    %0.4f       %0.4f    %0.4f   %0.4f   %0.4f" % (AUC, Acc,Recal,Prec,F1, E))
    
    AUC, Acc, Recal, Prec, F1, E =  crossCheck(RF, X_train, X_test, y_train, y_test)
    print("Random Forest:  %0.4f    %0.4f       %0.4f    %0.4f   %0.4f   %0.4f" % (AUC, Acc,Recal,Prec,F1, E))

    AUC, Acc, Recal, Prec, F1 , E =  crossCheck(GBI, X_train, X_test, y_train, y_test)
    print("Grad. Bosting:  %0.4f    %0.4f       %0.4f    %0.4f   %0.4f   %0.4f" % (AUC, Acc,Recal,Prec,F1, E))

    AUC, Acc, Recal, Prec, F1 , E =  crossCheck(NB, X_train, X_test, y_train, y_test)
    print("Naive Bayes :   %0.4f    %0.4f       %0.4f    %0.4f   %0.4f   %0.4f" % (AUC, Acc,Recal,Prec,F1, E))

    AUC, Acc, Recal, Prec, F1 , E =  crossCheck(AB, X_train, X_test, y_train, y_test)
    print("Ada.. Boost :   %0.4f    %0.4f       %0.4f    %0.4f   %0.4f   %0.4f" % (AUC, Acc,Recal,Prec,F1, E))

    
    AUC, Acc, Recal, Prec, F1 , E =  crossCheck(XG, X_train, X_test, y_train, y_test)
    print("XGrad Boost :   %0.4f    %0.4f       %0.4f    %0.4f   %0.4f   %0.4f" % (AUC, Acc,Recal,Prec,F1, E))


    AUC, Acc, Recal, Prec, F1 , E =  crossCheck(LG, X_train, X_test, y_train, y_test)
    print("LightGBM... :   %0.4f    %0.4f       %0.4f    %0.4f   %0.4f   %0.4f" % (AUC, Acc,Recal,Prec,F1, E))


    AUC, Acc, Recal, Prec, F1 , E =  crossCheck(CB, X_train, X_test, y_train, y_test)
    print("Cat Boost.. :   %0.4f    %0.4f       %0.4f    %0.4f   %0.4f   %0.4f" % (AUC, Acc,Recal,Prec,F1, E))



In [6]:
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [7]:
missing_values_table(data)

Your selected dataframe has 65 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [ ]:
# Prepare data for modeling

y = data.Label
X = data.drop('Label', axis=1) #Removing the label column

X = X.astype('float32')

from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=3)
X = pd.DataFrame(imputer.fit_transform(X))



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25, random_state=342) #Stem num

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((30750, 64), (10251, 64), (30750,), (10251,))

In [ ]:
classifiers()

Classifier:    AUC        Accuracy     Recall   Prec     F1     time-Elapsed
-----------------------------------------------------------------------------
Logistric Reg:  0.6610    0.9270       0.0948    0.9630   0.1726   0.0046
Decision Tree:  0.9880    0.9962       0.9781    0.9746   0.9763   0.0030
Random Forest:  0.9990    0.9923       0.9052    0.9987   0.9496   0.0875
Grad. Bosting:  0.9583    0.9822       0.7813    0.9969   0.8760   0.0128
Naive Bayes :   0.7668    0.9417       0.3645    0.8000   0.5008   0.0140
Ada.. Boost :   0.9986    0.9915       0.9137    0.9792   0.9453   0.0700
XGrad Boost :   0.9992    0.9991       0.9915    0.9976   0.9945   0.0115
LightGBM... :   0.9990    0.9995       0.9964    0.9976   0.9970   0.0251
0:	learn: 0.1874491	total: 199ms	remaining: 794ms
1:	learn: 0.0969271	total: 216ms	remaining: 324ms
2:	learn: 0.0575389	total: 229ms	remaining: 153ms
3:	learn: 0.0380152	total: 238ms	remaining: 59.6ms
4:	learn: 0.0243730	total: 249ms	remaining: 0us
Cat 

# Feature Selector

In [ ]:
clf = RandomForestClassifier(n_jobs=-1)
from mlxtend.feature_selection import SequentialFeatureSelector

# Build step forward feature selection
feature_selector = SequentialFeatureSelector(clf,
                                             k_features=64,
                                             forward=True,
                                             floating=False,
                                             verbose=2,
                                             scoring='accuracy',
                                             cv=5,
                                             n_jobs=6
                                            )

features = feature_selector.fit(X, y)

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:   55.0s
[Parallel(n_jobs=6)]: Done  64 out of  64 | elapsed:  1.5min finished

[2022-05-14 12:24:59] Features: 1/64 -- score: 0.9323907197516068[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:  1.1min
[Parallel(n_jobs=6)]: Done  63 out of  63 | elapsed:  2.1min finished

[2022-05-14 12:27:02] Features: 2/64 -- score: 0.9369025431163956[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:   55.2s
[Parallel(n_jobs=6)]: Done  62 out of  62 | elapsed:  1.8min finished

[2022-05-14 12:28:50] Features: 3/64 -- score: 0.9367561540680643[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:  1.0min
[Parallel(n_jobs=6)]: Done  61 out of  61 | elap

In [ ]:
clf = RandomForestClassifier(n_jobs=-1)
from mlxtend.feature_selection import SequentialFeatureSelector

# Build step forward feature selection
feature_selector = SequentialFeatureSelector(clf,
                                             k_features=24,
                                             forward=True,
                                             floating=False,
                                             verbose=2,
                                             scoring='accuracy',
                                             cv=5,
                                             n_jobs=6
                                            )

features = feature_selector.fit(X, y)


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:   54.6s
[Parallel(n_jobs=6)]: Done  64 out of  64 | elapsed:  1.4min finished

[2022-05-14 14:23:21] Features: 1/24 -- score: 0.9327565406955131[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:  1.0min
[Parallel(n_jobs=6)]: Done  63 out of  63 | elapsed:  2.0min finished

[2022-05-14 14:25:21] Features: 2/24 -- score: 0.9365611005201627[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:   55.5s
[Parallel(n_jobs=6)]: Done  62 out of  62 | elapsed:  1.8min finished

[2022-05-14 14:27:07] Features: 3/24 -- score: 0.9369025520385676[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:  1.1min
[Parallel(n_jobs=6)]: Done  61 out of  61 | elap

In [ ]:
filtered_features= X_train.columns[list(features.k_feature_idx_)]
print(filtered_features)

X_train = X_train[filtered_features]
X_test = X_test[filtered_features]

Int64Index([ 8, 10, 11, 13, 21, 22, 30, 34, 36, 37, 38, 39, 40, 41, 42, 43, 44,
            45, 46, 47, 49, 50, 57, 62],
           dtype='int64')


In [ ]:
classifiers()

Classifier:    AUC        Accuracy     Recall   Prec     F1     time-Elapsed
-----------------------------------------------------------------------------
Logistric Reg:  0.8622    0.9623       0.5529    0.9619   0.7022   0.0021
Decision Tree:  0.9016    0.9709       0.8190    0.8190   0.8190   0.0024
Random Forest:  0.9708    0.9801       0.7521    1.0000   0.8585   0.1004
Grad. Bosting:  0.9425    0.9669       0.5942    0.9899   0.7426   0.0093
Naive Bayes :   0.7382    0.9352       0.2139    0.9119   0.3465   0.0068
Ada.. Boost :   0.9581    0.9688       0.6318    0.9683   0.7647   0.0525
XGrad Boost :   0.9885    0.9836       0.8092    0.9838   0.8880   0.0085
LightGBM... :   0.9850    0.9826       0.7947    0.9864   0.8802   0.0141
0:	learn: 0.2025776	total: 32ms	remaining: 128ms
1:	learn: 0.1202329	total: 37.1ms	remaining: 55.6ms
2:	learn: 0.1021622	total: 42.3ms	remaining: 28.2ms
3:	learn: 0.0937457	total: 48.4ms	remaining: 12.1ms
4:	learn: 0.0867797	total: 54.2ms	remaining: 0us

In [ ]:
from imblearn.under_sampling import TomekLinks, ClusterCentroids
# UnderSample with TomekLinks

_TomekLinks = TomekLinks()

X_res, y_res = _TomekLinks.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split (X_res, y_res, test_size=0.25, random_state=342)

In [ ]:
classifiers()

Classifier:    AUC        Accuracy     Recall   Prec     F1     time-Elapsed
-----------------------------------------------------------------------------
Logistric Reg:  0.5798    0.9272       0.0846    0.9853   0.1558   0.0043
Decision Tree:  0.9949    0.9981       0.9912    0.9849   0.9880   0.0026
Random Forest:  0.9994    0.9928       0.9104    0.9986   0.9524   0.0835
Grad. Bosting:  0.9606    0.9830       0.7891    0.9952   0.8803   0.0118
Naive Bayes :   0.7630    0.9412       0.3485    0.7954   0.4846   0.0154
Ada.. Boost :   0.9978    0.9912       0.8965    0.9916   0.9416   0.0694
XGrad Boost :   1.0000    0.9991       0.9912    0.9975   0.9943   0.0091
LightGBM... :   1.0000    0.9993       0.9924    0.9987   0.9956   0.0193
0:	learn: 0.1946097	total: 30.3ms	remaining: 121ms
1:	learn: 0.0942572	total: 40.9ms	remaining: 61.4ms
2:	learn: 0.0585005	total: 52ms	remaining: 34.6ms
3:	learn: 0.0386689	total: 63ms	remaining: 15.8ms
4:	learn: 0.0278574	total: 73.2ms	remaining: 0us
C

In [ ]:
# UnderSample with ClusterCentroids
from imblearn.under_sampling import TomekLinks, ClusterCentroids

_ClusterCentroids = ClusterCentroids()

X_res, y_res = _ClusterCentroids.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split (X_res, y_res, test_size=0.25, random_state=342)

In [ ]:
classifiers()

Classifier:    AUC        Accuracy     Recall   Prec     F1     time-Elapsed
-----------------------------------------------------------------------------
Logistric Reg:  0.5295    0.5665       0.4184    0.5900   0.4896   0.0017
Decision Tree:  0.9878    0.9878       0.9926    0.9830   0.9878   0.0020
Random Forest:  0.9998    0.9909       0.9963    0.9854   0.9908   0.0215
Grad. Bosting:  0.9991    0.9890       0.9975    0.9807   0.9891   0.0040
Naive Bayes :   0.7266    0.6451       0.3006    0.9533   0.4571   0.0019
Ada.. Boost :   0.9989    0.9909       0.9963    0.9854   0.9908   0.0177
XGrad Boost :   0.9999    0.9927       0.9975    0.9878   0.9927   0.0052
LightGBM... :   0.9999    0.9933       0.9951    0.9914   0.9933   0.0037
0:	learn: 0.1481329	total: 38ms	remaining: 152ms
1:	learn: 0.0768630	total: 46ms	remaining: 69ms
2:	learn: 0.0544586	total: 53ms	remaining: 35.4ms
3:	learn: 0.0440080	total: 59.3ms	remaining: 14.8ms
4:	learn: 0.0340577	total: 65.3ms	remaining: 0us
Cat B

In [ ]:
# OverSample with SMOTE
from imblearn.over_sampling import ADASYN, SMOTE

_smote = SMOTE(random_state=42)

X_res, y_res = _smote.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split (X_res, y_res, test_size=0.25, random_state=342)

In [ ]:
classifiers()

Classifier:    AUC        Accuracy     Recall   Prec     F1     time-Elapsed
-----------------------------------------------------------------------------
Logistric Reg:  0.5862    0.6351       0.4621    0.7084   0.5594   0.0090
Decision Tree:  0.9980    0.9980       0.9979    0.9982   0.9980   0.0037
Random Forest:  1.0000    0.9994       0.9992    0.9997   0.9994   0.1757
Grad. Bosting:  0.9976    0.9769       0.9628    0.9909   0.9766   0.0338
Naive Bayes :   0.7717    0.6981       0.4545    0.8887   0.6014   0.0282
Ada.. Boost :   0.9986    0.9835       0.9779    0.9891   0.9835   0.1302
XGrad Boost :   0.9999    0.9998       0.9996    1.0000   0.9998   0.0187
LightGBM... :   1.0000    0.9997       0.9994    1.0000   0.9997   0.0409
0:	learn: 0.2854354	total: 48ms	remaining: 192ms
1:	learn: 0.1398375	total: 62.6ms	remaining: 93.9ms
2:	learn: 0.0918432	total: 80.9ms	remaining: 53.9ms
3:	learn: 0.0643454	total: 96.4ms	remaining: 24.1ms
4:	learn: 0.0480930	total: 111ms	remaining: 0us


In [ ]:
# OverSample with ADASYN
from imblearn.over_sampling import ADASYN, SMOTE

_adasyn = ADASYN()

X_res, y_res = _adasyn.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split (X_res, y_res, test_size=0.25, random_state=342)

In [ ]:
classifiers()

Classifier:    AUC        Accuracy     Recall   Prec     F1     time-Elapsed
-----------------------------------------------------------------------------
Logistric Reg:  0.5713    0.5434       0.5280    0.5483   0.5380   0.0087
Decision Tree:  0.9981    0.9982       0.9984    0.9979   0.9982   0.0037
Random Forest:  1.0000    0.9990       0.9987    0.9993   0.9990   0.1810
Grad. Bosting:  0.9989    0.9794       0.9621    0.9968   0.9792   0.0350
Naive Bayes :   0.6885    0.6083       0.7341    0.5891   0.6536   0.0285
Ada.. Boost :   0.9984    0.9826       0.9785    0.9868   0.9826   0.1293
XGrad Boost :   1.0000    0.9996       0.9994    0.9999   0.9996   0.0152
LightGBM... :   1.0000    0.9997       0.9995    1.0000   0.9997   0.0305
0:	learn: 0.2868581	total: 17.7ms	remaining: 70.6ms
1:	learn: 0.1579722	total: 30.1ms	remaining: 45.1ms
2:	learn: 0.0983857	total: 41.7ms	remaining: 27.8ms
3:	learn: 0.0705381	total: 54.4ms	remaining: 13.6ms
4:	learn: 0.0519454	total: 68.8ms	remaining: 